In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from datetime import datetime, timedelta

In [ ]:
df = pd.read_csv("../files/students.csv", index_col = 0)

# Pair Programming ETL Transformación II

En la lección de hoy aprendimos como Crearnos una clase que nos permita limpiar los datos obtenidos de la API.

En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I y II.